<a href="https://colab.research.google.com/github/nianlonggu/MemSum/blob/main/oracle_score_compute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf ./*
!git clone https://github.com/nianlonggu/MemSum.git

Cloning into 'MemSum'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 224 (delta 53), reused 54 (delta 9), pack-reused 98
Receiving objects: 100% (224/224), 81.90 MiB | 23.38 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [2]:
import os
os.chdir("MemSum")

In [3]:
!pip install -r requirements.txt -q
!pip install gdown -q

!gdown 1nNn1h9pz6JdelksMn5xh2zVoek1oh3GF

from rouge_score.scoring import BootstrapAggregator
from rouge_score import rouge_scorer
from tqdm import tqdm
from src.data_preprocessing.MemSum.utils import greedy_extract
import json


scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeLsum'], use_stemmer = True)
aggregator = BootstrapAggregator()


     |████████████████████████████████| 60 kB 1.8 MB/s 
Downloading...
From: https://drive.google.com/uc?id=1nNn1h9pz6JdelksMn5xh2zVoek1oh3GF
To: /content/MemSum/test_PUBMED.jsonl
100% 89.9M/89.9M [00:00<00:00, 109MB/s]


In [4]:
#use bootstrapping
#test_PUBMED is from the trunc dataset
with open('test_PUBMED.jsonl') as f:
    for count, line in enumerate(tqdm(f)):
        data = json.loads(line)
        oracle_indices = greedy_extract(data['text'],data['summary'])[0][0]
        oracle_summary = [data['text'][idx] for idx in oracle_indices]

        aggregator.add_scores(scorer.score('\n'.join(data['summary']),
                                           '\n'.join(oracle_summary)
                                         ))
res = aggregator.aggregate()

5025it [05:46, 14.50it/s]


In [5]:
res

{'rouge1': AggregateScore(low=Score(precision=0.530096094981509, recall=0.449312050302839, fmeasure=0.47395020218708467), mid=Score(precision=0.5337434753244591, recall=0.45280112214401325, fmeasure=0.4767111464972332), high=Score(precision=0.5372552992377749, recall=0.4561009790359482, fmeasure=0.47943544311157615)),
 'rouge2': AggregateScore(low=Score(precision=0.22838795986920768, recall=0.18630491408643465, fmeasure=0.1988434122131042), mid=Score(precision=0.2325194699725009, recall=0.18917402891102902, fmeasure=0.2017826917293284), high=Score(precision=0.23676558206224854, recall=0.1921165072575005, fmeasure=0.20494078438495247)),
 'rougeLsum': AggregateScore(low=Score(precision=0.4714029899544923, recall=0.39674843527780673, fmeasure=0.4196365612134437), mid=Score(precision=0.4753419599077294, recall=0.3998085164891515, fmeasure=0.42236794085218), high=Score(precision=0.4790487187082761, recall=0.40296195717795, fmeasure=0.42526701118383464))}